In [ ]:
# external imports
import os
import re
import pickle
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
from matplotlib.patches import Rectangle
from pymatgen.core.structure import Structure
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

# setup for plots
plt.rcParams["text.usetex"] = True
plt.rc("font", family="serif")
plt.rc("xtick", labelsize="x-small")
plt.rc("ytick", labelsize="x-small")
plt.rc("text", usetex=True)

# so python can find the calc directory
os.chdir("../")

The state-of-the-art (SOTA) algorithm mentioned in the paper is labeled
"npj" instead of "sota" in the code for historical reasons.

In [ ]:
# data structures that contain all relevant information
@dataclass
class txt_data:
    id: str  # materials project id
    name: str  # material name
    structure: Structure  # pymatgen structure class
    data: list  # output of the .txt file
    gwc: list  # list with all output classes


@dataclass
class cdata:
    id: str  # materials project id
    name: str  # material name
    structure: Structure  # pymatgen structure class
    kppa: np.array  # k-point density
    kpt_str: list  # k-point string
    nkir: np.array  # number of irreducible k-points
    params: np.ndarray  # gw parameters
    gap: np.array  # direct gap at the gamma point
    ngw: np.ndarray  # total number of gw calculations
    tcpu: np.array  # cpu time
    gwc: list  # list with all output classes
    extra: np.array  # gap extrapolation from the fit algorithm

In [ ]:
# parse my txt files
def output_parser(txt_path, calc_type):
    with open(txt_path, "r") as f:
        s = f.readlines()
    s = s[1:]  # skip the header
    if calc_type == "cs":
        data = np.array(-1 * np.ones([5, 10], dtype=int), dtype=str)
    elif calc_type == "npj":
        data = np.array(-1 * np.ones([5, 11], dtype=int), dtype=str)
    elif calc_type == "ref":
        data = np.array(-1 * np.ones([5, 9], dtype=int), dtype=str)
    for i, l in enumerate(s):
        if not "FAILED" in l:
            data[i] = l.split()
    return data

In [ ]:
"""
Parse the results for all 60 bulk materials
"""

# initializations
calc_dir = "calc"
mp_ids = os.listdir(calc_dir)
materials_2d = os.listdir("input_2d")
materials_2d = [x.split(".")[0] for x in materials_2d]
[mp_ids.remove(f) for f in materials_2d]
res_cs = []
res_npj = []
res_ref = []
conv_thr = 25  # meV

# to satisfy my own OCD we sort by the mp-ids
sort_idx = np.argsort([int(x.split("-")[1]) for x in mp_ids])
mp_ids = np.array(mp_ids)[sort_idx]

# loop over all materials
for mp in mp_ids:

    # materials project id
    id = re.findall(r"mp-\d+", mp)[0]

    # parse the structure
    with open(f"{calc_dir}/{mp}/structure.pckl", "rb") as f:
        structure, name = pickle.load(f)
    for site in structure.sites:  # fix the site labels ...
        site.label = site.species.elements[0].name

    # collect the data from the cooridnate search algorithm
    cs_path = f"{calc_dir}/{mp}/yambo_g0w0_cs_kpt"
    if os.path.isfile(f"{cs_path}/{mp}_gw_conv_cs.txt"):
        # data from the output text file
        data = output_parser(f"{cs_path}/{mp}_gw_conv_cs.txt", "cs")

        # collect all classes
        cs_class = []
        for kppa in data[:, 0]:
            cs_class_path = f"{cs_path}/kppa{kppa}_cs/g0w0_cs/class_cs.pckl"
            if os.path.isfile(cs_class_path):
                with open(cs_class_path, "rb") as f:
                    gw_class = pickle.load(f)
            else:
                gw_class = 0
            cs_class.append(gw_class)

        # create the dataclass for each k-grid
        res_cs.append(
            cdata(
                id,  # materials project id
                name,  # material name
                structure,  # pymatgen structure class
                data[:, 0].astype(int),  # kppa
                data[:, 1],  # k-point string
                data[:, 2].astype(int),  # number of irreducible k-points
                data[:, 3:6].astype(int),  # gw parameteres
                data[:, 6].astype(float),  # direct gap
                data[:, 7].astype(int),  # number of gw calculations
                data[:, 8].astype(int) * data[:, 9].astype(int),  # cpu time
                cs_class,  # list with all classes
                np.zeros(5),  # gap extrapolation from the fit algorithm
            )
        )

    else:
        print(f"{mp} cs is missing...")

    # collect the data from the npj fit algorithm
    npj_path = f"{calc_dir}/{mp}/yambo_g0w0_npj_kpt/"
    if os.path.isfile(f"{npj_path}/{mp}_gw_conv_npj.txt"):
        # data from the output text file
        data = output_parser(f"{npj_path}/{mp}_gw_conv_npj.txt", "npj")

        # collect all classes
        npj_class = []
        extra = []
        for kppa in data[:, 0]:
            npj_class_path = f"{npj_path}/kppa{kppa}_npj/g0w0_npj/class_npj.pckl"
            if os.path.isfile(npj_class_path):
                with open(npj_class_path, "rb") as f:
                    gw_class = pickle.load(f)
            else:
                gw_class = 0
            npj_class.append(gw_class)
            if gw_class == 0:
                extra.append(-1)
            else:
                extra.append(gw_class.extra)
        extra = np.array(extra)

        # create the dataclass for each k-grid
        res_npj.append(
            cdata(
                id,  # materials project id
                name,  # material name
                structure,  # pymatgen structure class
                data[:, 0].astype(int),  # kppa
                data[:, 1],  # k-point string
                data[:, 2].astype(int),  # number of irreducible k-points
                data[:, 3:6].astype(int),  # gw parameteres
                data[:, 6].astype(float),  # direct gap
                data[:, 7].astype(int),  # number of gw calculations
                data[:, 8].astype(int) * data[:, 9].astype(int),  # cpu time
                npj_class,  # list with all classes
                extra,  # gap extrapolation from the fit algorithm
            )
        )
    else:
        print(f"{mp} npj is missing...")

    # collect the data from the cs reference calculations with high computation parameters
    ref_path = f"{calc_dir}/{mp}/yambo_g0w0_cs_reference/"
    if os.path.isfile(f"{ref_path}/{mp}_gw_conv_cs_ref.txt"):
        # data from the output text file
        data = output_parser(f"{ref_path}/{mp}_gw_conv_cs_ref.txt", "ref")

        # create the dataclass for each k-grid
        res_ref.append(
            cdata(
                id,  # materials project id
                name,  # material name
                structure,  # pymatgen structure class
                data[:, 0].astype(int),  # kppa
                data[:, 1],  # k-point string
                data[:, 2].astype(int),  # number of irreducible k-points
                data[:, 3:6].astype(int),  # gw parameters
                data[:, 6].astype(float),  # direct gap
                np.ones(5),  # number of gw calculations
                data[:, 7].astype(int) * data[:, 8].astype(int),  # cpu time
                [],  # list with all classes
                np.zeros(5),  # gap extrapolation from the fit algorithm
            )
        )
    else:
        print(f"{mp} ref is missing...")

# number of finished calculations
print(f"#cs  = {len(res_cs)}")
print(f"#npj = {len(res_npj)}")
print(f"#ref = {len(res_ref)}")

In [ ]:
# some statistics
total_gw = 0
total_tcpu = 0
for i in range(len(res_cs)):
    idx_used_cs = res_cs[i].ngw > 0
    idx_used_npj = res_cs[i].ngw > 0
    total_gw += np.sum(res_cs[i].ngw[idx_used_cs])
    total_gw += np.sum(res_npj[i].ngw[idx_used_npj])
    total_gw += np.sum(res_ref[i].ngw)
    total_tcpu += np.sum(res_cs[i].tcpu[idx_used_cs])
    total_tcpu += np.sum(res_npj[i].tcpu[idx_used_npj])
    total_tcpu += np.sum(res_ref[i].tcpu)
print(f"Total number of GW calculation for all plots: {int(total_gw):d}!")
print(
    f"Total CPU time for all plots: {np.round(total_tcpu / 3600 / 24 / 365, 1):.1f} years!"
)
print("This includes the reference calculations!")

In [ ]:
# average gap difference in meV, cpu time and number of gw calculations
# for each k-grid with the same number of irreducible k-points as well
# as the computation time needed for each convergence calculation
num_mat = len(res_cs)
num_kpt = len(res_cs[0].kppa)
gap_diff_cs = []
ngw_cs = []
time_cs = []
gap_diff_npj = []
ngw_npj = []
time_npj = []
for i in range(num_mat):
    for j in range(num_kpt):
        if (res_cs[i].nkir[j] > 0) and (res_npj[i].nkir[j] > 0):
            temp_gap_diff_cs = np.abs(res_cs[i].gap[j] - res_ref[i].gap[j]) * 1000
            if temp_gap_diff_cs > 50:
                print(
                    f"CS Large gap difference:  {res_cs[i].id:<10s} {res_cs[i].name:<8s} kppa = {res_cs[i].kppa[j]:<5d} gap_diff = {temp_gap_diff_cs:.2f} meV"
                )
            gap_diff_cs.append([res_cs[i].nkir[j], temp_gap_diff_cs])
            if res_cs[i].ngw[j] > 11:
                print(
                    f"CS Large number of GW:    {res_cs[i].id:<10s} {res_cs[i].name:<8s} kppa = {res_cs[i].kppa[j]:<5d} ngw = {res_cs[i].ngw[j]:d}"
                )
            ngw_cs.append([res_cs[i].nkir[j], res_cs[i].ngw[j]])
            time_cs.append([res_cs[i].nkir[j], res_cs[i].tcpu[j]])
            temp_gap_diff_npj = np.abs(res_npj[i].gap[j] - res_ref[i].gap[j]) * 1000
            if temp_gap_diff_npj > 50:
                print(
                    f"NPJ Large gap difference: {res_cs[i].id:<10s} {res_cs[i].name:<8s} kppa = {res_npj[i].kppa[j]:<5d} gap_diff = {temp_gap_diff_npj:.2f} meV"
                )
            gap_diff_npj.append([res_npj[i].nkir[j], temp_gap_diff_npj])
            if res_npj[i].ngw[j] > 35:
                print(
                    f"NPJ Large number of GW:   {res_npj[i].id:<10s} {res_npj[i].name:<8s} kppa = {res_npj[i].kppa[j]:<5d} ngw = {res_npj[i].ngw[j]:d}"
                )
            ngw_npj.append([res_npj[i].nkir[j], res_npj[i].ngw[j]])
            time_npj.append([res_npj[i].nkir[j], res_npj[i].tcpu[j]])
            if res_cs[i].tcpu[j] > res_npj[i].tcpu[j]:
                print(
                    f"CS slower than NPJ:       {res_cs[i].id:<10s} {res_cs[i].name:<8s} kppa = {res_cs[i].kppa[j]:<5d} speedup = {res_npj[i].tcpu[j]/res_cs[i].tcpu[j]:.2f}"
                )
        else:
            print(
                f"Skipping: {res_cs[i].id:<10s} {res_cs[i].name:<8s} kppa = {res_cs[i].kppa[j]:d}"
            )
gap_diff_cs = np.array(gap_diff_cs)
ngw_cs = np.array(ngw_cs)
time_cs = np.array(time_cs)
gap_diff_npj = np.array(gap_diff_npj)
ngw_npj = np.array(ngw_npj)
time_npj = np.array(time_npj)

# we ignore the data for both algorithms for a material at a specific k-point grid
# in our analysis if one of the algorithms failed to converge for this specific point
# (only the npj/sota algorithm failed in same test cases, while the cs always worked)

Compare the accuracy and performance of both algorithms.

In [ ]:
# figure size (tested)
width = 2 * 2.65
height = 4.5

# text size
fs = 10

# figure
fig, axs = plt.subplots(2, 2, figsize=(width, height), facecolor="w", dpi=600)

# add (a), (b), (c) and (d) to the subplots
axs[0, 0].text(-0.25, 1.15, r"\textbf{a}", transform=axs[0, 0].transAxes, size=11)
axs[0, 1].text(-0.25, 1.15, r"\textbf{b}", transform=axs[0, 1].transAxes, size=11)
axs[1, 0].text(-0.25, 1.15, r"\textbf{c}", transform=axs[1, 0].transAxes, size=11)
axs[1, 1].text(-0.25, 1.15, r"\textbf{d}", transform=axs[1, 1].transAxes, size=11)

# plot the gap differences
axs[0, 0].hist(gap_diff_cs[:, 1], bins=50, color="tab:blue")
axs[0, 0].axvline(x=conv_thr, color="k", linestyle="-")
axs[0, 0].axvline(x=np.mean(gap_diff_cs[:, 1]), color="magenta", linestyle="--")
axs[0, 0].axvline(x=np.median(gap_diff_cs[:, 1]), color="limegreen", linestyle="--")
axs[0, 1].hist(gap_diff_npj[:, 1], bins=50, color="tab:blue")
axs[0, 1].axvline(x=conv_thr, color="k", linestyle="-")
axs[0, 1].axvline(x=np.mean(gap_diff_npj[:, 1]), color="magenta", linestyle="--")
axs[0, 1].axvline(x=np.median(gap_diff_npj[:, 1]), color="limegreen", linestyle="--")

# plot the number of gw calculations
axs[1, 0].hist(
    ngw_cs[:, 1],
    bins=np.arange(np.min(ngw_cs[:, 1]), np.max(ngw_cs[:, 1]) + 5, 1) - 0.5,
    color="tab:blue",
)
axs[1, 0].axvline(x=np.mean(ngw_cs[:, 1]), color="magenta", linestyle="--")
axs[1, 0].axvline(x=np.median(ngw_cs[:, 1]), color="limegreen", linestyle="--")
axs[1, 0].axvspan(
    0, 14, color="silver", zorder=-1, edgecolor=None
)  # highlight inset area in the main plot
axs[1, 1].hist(
    ngw_npj[:, 1],
    bins=np.arange(np.min(ngw_npj[:, 1]), np.max(ngw_npj[:, 1]) + 5, 1) - 0.5,
    color="tab:blue",
)
axs[1, 1].axvline(x=np.mean(ngw_npj[:, 1]), color="magenta", linestyle="--")
axs[1, 1].axvline(x=np.median(ngw_npj[:, 1]), color="limegreen", linestyle="--")

# inset for the number of CS GW calculations
axins = inset_axes(
    axs[1, 0],
    width=1.0,
    height=0.8,
    bbox_to_anchor=(0.275, 0.175, 0.6, 0.2),
    bbox_transform=axs[1, 0].transAxes,
    loc=3,
)
axins.hist(
    ngw_cs[:, 1],
    bins=np.arange(np.min(ngw_cs[:, 1]), np.max(ngw_cs[:, 1]) + 5, 1) - 0.5,
    color="tab:blue",
)
axins.axvline(x=np.mean(ngw_cs[:, 1]), color="magenta", linestyle="--")
axins.axvline(x=np.median(ngw_cs[:, 1]), color="limegreen", linestyle="--")
axins.xaxis.set_tick_params(labelsize=fs - 4)
axins.yaxis.set_tick_params(labelsize=fs - 4)
axins.tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=2,
    direction="in",
)
axins.tick_params(labelbottom=True, labeltop=False, labelleft=False, labelright=True)
axins.set_xlim([0, 14])
axins.set_ylim([0, 90])
axins.xaxis.set_ticks([2, 4, 6, 8, 10, 12])
axins.yaxis.set_ticks([0, 25, 50, 75])

# axis labels
axs[0, 0].set_xlabel(
    r"$\vert\Delta_\mathrm{R}\mathrm{E}_\mathrm{gap}^{{\mathbf{\Gamma}}-{\mathbf{\Gamma}}}\vert$ (meV)",
    size=fs,
)
axs[0, 1].set_xlabel(
    r"$\vert\Delta_\mathrm{R}\mathrm{E}_\mathrm{gap}^{{\mathbf{\Gamma}}-{\mathbf{\Gamma}}}\vert$ (meV)",
    size=fs,
)
axs[0, 0].set_ylabel(r"Occurrences", size=fs)
axs[1, 0].set_xlabel(r"$N_\mathrm{GW}$", size=fs)
axs[1, 1].set_xlabel(r"$N_\mathrm{GW}$", size=fs)
axs[1, 0].set_ylabel(r"Occurrences", size=fs)

# titles
axs[0, 0].set_title(r"CS", fontsize=fs + 2, pad=15)
axs[0, 1].set_title(r"SOTA", fontsize=fs + 2, pad=15)

# axis ticks
axs[0, 0].xaxis.set_tick_params(labelsize=fs)
axs[0, 0].yaxis.set_tick_params(labelsize=fs)
axs[0, 0].tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=4,
    direction="in",
)
axs[0, 0].tick_params(
    labelbottom=True, labeltop=False, labelleft=True, labelright=False
)
axs[0, 0].set_xlim([0, 250])
axs[0, 0].xaxis.set_ticks([0, 50, 100, 150, 200, 250])
axs[0, 0].yaxis.set_ticks(axs[0, 0].get_yticks().astype(int))
axs[0, 1].xaxis.set_tick_params(labelsize=fs)
axs[0, 1].yaxis.set_tick_params(labelsize=fs)
axs[0, 1].tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=4,
    direction="in",
)
axs[0, 1].tick_params(
    labelbottom=True, labeltop=False, labelleft=True, labelright=False
)
axs[0, 1].set_xlim([0, 250])
axs[0, 1].xaxis.set_ticks([0, 50, 100, 150, 200, 250])
axs[0, 1].yaxis.set_ticks(axs[0, 1].get_yticks().astype(int))
axs[1, 0].xaxis.set_tick_params(labelsize=fs)
axs[1, 0].yaxis.set_tick_params(labelsize=fs)
axs[1, 0].tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=4,
    direction="in",
)
axs[1, 0].tick_params(
    labelbottom=True, labeltop=False, labelleft=True, labelright=False
)
axs[1, 0].set_xlim([0, 75])
axs[1, 0].xaxis.set_ticks([0, 10, 20, 30, 40, 50, 60, 70])
axs[1, 0].yaxis.set_ticks([0, 20, 40, 60, 80])
axs[1, 1].xaxis.set_tick_params(labelsize=fs)
axs[1, 1].yaxis.set_tick_params(labelsize=fs)
axs[1, 1].tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=4,
    direction="in",
)
axs[1, 1].tick_params(
    labelbottom=True, labeltop=False, labelleft=True, labelright=False
)
axs[1, 1].set_xlim([0, 75])
axs[1, 1].xaxis.set_ticks([0, 10, 20, 30, 40, 50, 60, 70])
axs[1, 1].yaxis.set_ticks([0, 20, 40, 60, 80, 100, 120])
axs[1, 1].set_xlim([1, np.max(ngw_npj[:, 1]) + 1])

# annotations
# highlight some outliers/interesting materials...
axs[0, 0].annotate(
    r"Kr, BN, AgI",
    xy=(60, 4),
    xycoords="data",
    xytext=(0, 45),
    textcoords="offset points",
    arrowprops=dict(arrowstyle="->"),
    fontsize=fs - 2,
)
axs[0, 0].annotate(
    r"KF, As$_2$Os",
    xy=(165, 2),
    xycoords="data",
    xytext=(-30, 30),
    textcoords="offset points",
    arrowprops=dict(arrowstyle="->"),
    fontsize=fs - 2,
)
axs[0, 0].annotate(
    r"BaF$_2$",
    xy=(195, 1),
    xycoords="data",
    xytext=(0, 15),
    textcoords="offset points",
    arrowprops=dict(arrowstyle="->"),
    fontsize=fs - 2,
)
axs[0, 1].annotate(
    r"AlSb, BaO",
    xy=(53, 9),
    xycoords="data",
    xytext=(-10, 40),
    textcoords="offset points",
    arrowprops=dict(arrowstyle="->"),
    fontsize=fs - 2,
)
axs[0, 1].annotate(
    r"BaF$_2$",
    xy=(75, 4),
    xycoords="data",
    xytext=(10, 20),
    textcoords="offset points",
    arrowprops=dict(arrowstyle="->"),
    fontsize=fs - 2,
)
axs[0, 1].annotate(
    r"AgI, As$_2$Os",
    xy=(156.5, 4),
    xycoords="data",
    xytext=(-20, 35),
    textcoords="offset points",
    arrowprops=dict(arrowstyle="->"),
    fontsize=fs - 2,
)
axs[0, 1].annotate(
    r"ZnO",
    xy=(224, 1),
    xycoords="data",
    xytext=(-20, 20),
    textcoords="offset points",
    arrowprops=dict(arrowstyle="->"),
    fontsize=fs - 2,
)
axs[1, 1].annotate(
    "C, YN,\n" + r"Li$_2$O",
    xy=(35, 4),
    xycoords="data",
    xytext=(-25, 25),
    textcoords="offset points",
    arrowprops=dict(arrowstyle="->"),
    fontsize=fs - 2,
)
axs[1, 1].annotate(
    "Kr, LiH,\n" + "NaF, BN",
    xy=(42, 3),
    xycoords="data",
    xytext=(-20, 50),
    textcoords="offset points",
    arrowprops=dict(arrowstyle="->"),
    fontsize=fs - 2,
)
axs[1, 1].annotate(
    r"CsCl",
    xy=(65, 1),
    xycoords="data",
    xytext=(-30, 30),
    textcoords="offset points",
    arrowprops=dict(arrowstyle="->"),
    fontsize=fs - 2,
)
axs[1, 1].annotate(
    r"AgCl",
    xy=(73, 1),
    xycoords="data",
    xytext=(-25, 20),
    textcoords="offset points",
    arrowprops=dict(arrowstyle="->"),
    fontsize=fs - 2,
)
axins.annotate(
    r"As$_2$Os," + "\nothers",
    xy=(3, 17),
    xycoords="data",
    xytext=(-12, 27),
    textcoords="offset points",
    arrowprops=dict(arrowstyle="->"),
    fontsize=fs - 2,
)
axins.annotate(
    "NaF,\n" + r"PbF$_2$",
    xy=(12, 2.5),
    xycoords="data",
    xytext=(-10, 25),
    textcoords="offset points",
    arrowprops=dict(arrowstyle="->"),
    fontsize=fs - 2,
)

# fix layout
fig.tight_layout()
fig.subplots_adjust(hspace=0.65, wspace=0.35)

# save the figure
fig.savefig("plots/paper/conv_analysis.pdf")

In [ ]:
# some statistics for the gap difference to the reference calculation
print("MEAN")
print(f"CS:   {np.mean(gap_diff_cs[:,1]):.2f} meV")
print(f"SOTA: {np.mean(gap_diff_npj[:,1]):.2f} meV\n")

print("MEDIAN")
print(f"CS:   {np.median(gap_diff_cs[:,1]):.2f} meV")
print(f"SOTA: {np.median(gap_diff_npj[:,1]):.2f} meV\n")

print("RANGE")
print(f"CS:   {np.max(gap_diff_cs[:,1]) - np.min(gap_diff_cs[:,1]):.2f} meV")
print(f"SOTA: {np.max(gap_diff_npj[:,1]) - np.min(gap_diff_npj[:,1]):.2f} meV")

In [ ]:
# some statistics for the number of gw calculations needed
print("MEAN")
print(f"CS:   {np.mean(ngw_cs[:,1]):.1f}")
print(f"SOTA: {np.mean(ngw_npj[:,1]):.1f}\n")

print("MEDIAN")
print(f"CS:   {np.median(ngw_cs[:,1]):.1f}")
print(f"SOTA: {np.median(ngw_npj[:,1]):.1f}\n")

print("RANGE")
print(f"CS:   {np.max(ngw_cs[:,1])-np.min(ngw_cs[:,1]):.1f}")
print(f"SOTA: {np.max(ngw_npj[:,1])-np.min(ngw_npj[:,1]):.1f}\n")

In [ ]:
# calculate the speed up
speed_up = time_npj[:, 1] / time_cs[:, 1]

# figure size (tested)
width = 2.65
height = 2.0

# text size
fs = 10

# figure
fig, ax = plt.subplots(1, 1, figsize=(width, height), facecolor="w", dpi=600)

# plot the relative time differences
ax.hist(
    speed_up,
    bins=np.arange(np.min(speed_up), np.max(speed_up), 0.1) - 0.5,
    color="tab:blue",
)
ax.axvline(x=1, color="k", linestyle="-")
ax.axvline(x=np.mean(speed_up), color="magenta", linestyle="--")
ax.axvline(x=np.median(speed_up), color="limegreen", linestyle="--")

# axis ticks
ax.xaxis.set_tick_params(labelsize=fs)
ax.yaxis.set_tick_params(labelsize=fs)
ax.tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=4,
    direction="in",
)
ax.tick_params(labelbottom=True, labeltop=False, labelleft=True, labelright=False)
ax.xaxis.set_ticks([0, 2, 4, 6, 8, 10])
ax.yaxis.set_ticks(ax.get_yticks().astype(int))

# axis limit
ax.set_xlim([0, 10])

# axis labels
ax.set_xlabel(r"$T_\mathrm{SOTA}/T_\mathrm{CS}$", size=fs)
ax.set_ylabel(r"Occurrences", size=fs)

# fix layout
fig.tight_layout()

# save the figure
fig.savefig("plots/paper/cs_speed_up.pdf")

In [ ]:
# some statistics for speed up
print(f"MEAN:   {np.mean(speed_up):.1f}")
print(f"MEDIAN: {np.median(speed_up):.1f}")
print(f"MAX:    {np.max(speed_up):.1f}")

In [ ]:
# compare the total computation times
print(f"Total CS computation time:   {np.sum(time_cs[:,1]) / 3600:.1f} h")
print(f"Total SOTA computation time: {np.sum(time_npj[:,1]) / 3600:.1f} h")

Plot comparing the convergence algorithms on the examples of Si and LiF for the most dense k-point grids.

In [ ]:
# redo the fits from the classes
# (they are deleted to reduce the file size of the output pickle file)
idx_conv_plt = [i for i in range(len(res_cs)) if res_cs[i].name in ["LiF", "Si"]]
for i in idx_conv_plt:
    res_npj[i].gwc[-1].get_best_fit()

In [ ]:
# figure size (tested)
width = 2 * 2.65
height = 3.0

# text size
fs = 10

# figure
fig, axes = plt.subplots(ncols=2, figsize=(width, height), facecolor="w", dpi=600)

# titles
axes[0].set_title("Si", fontsize=fs + 2, pad=15)
axes[1].set_title("LiF", fontsize=fs + 2, pad=15)

# add (a), (b) to the subplots
axes[0].text(-0.25, 1.15, r"\textbf{a}", transform=axes[0].transAxes, size=11)
axes[1].text(-0.25, 1.15, r"\textbf{b}", transform=axes[1].transAxes, size=11)

"""
Si SECTION
"""
# index for silicon
idx_si = idx_conv_plt[0]

# plot for the cs algorithm for Si
gw_class = res_cs[idx_si].gwc[-1]
axes[0].scatter(
    gw_class.grid[:, 0],
    gw_class.grid[:, 1],
    20,
    c="r",
    marker="x",
    zorder=1000,
)
axes[0].plot(gw_class.grid[:, 0], gw_class.grid[:, 1], "r-", linewidth=1, zorder=1000)
axes[0].scatter(
    gw_class.grid[-1, 0],
    gw_class.grid[-1, 1],
    75,
    c="r",
    marker="x",
    label=f"Converged = {np.round(gw_class.grid[-1,2],6):.3f} eV",
    zorder=20,
)
axes[0].xaxis.set_tick_params(labelsize=fs)
axes[0].yaxis.set_tick_params(labelsize=fs)
axes[0].set_xlabel(r"$N_\mathrm{b}$", fontsize=fs)
axes[0].set_ylabel(r"$G_\mathrm{cut}$ (Ry)", fontsize=fs)
axes[0].tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=4,
    direction="in",
)
axes[0].tick_params(labelbottom=True, labeltop=False, labelleft=True, labelright=False)

# plot the npj algorithm for Si
gw_class = res_npj[idx_si].gwc[-1]
ref_gap = res_ref[idx_si].gap[-1]
rel_error = 100 * abs(gw_class.z_fit - ref_gap) / ref_gap
abs_error = 1000 * abs(gw_class.z_fit - ref_gap)
fit = axes[0].scatter(
    gw_class.x_fit,
    gw_class.y_fit,
    30,
    c=abs_error,
    marker="s",
    zorder=10,
    cmap="viridis_r",
)
cmap = plt.colorbar(fit, shrink=0.75, aspect=30, pad=0.05)
cmap.ax.tick_params(labelsize=8)
axes[0].scatter(
    gw_class.grid_plot[:, 0],
    gw_class.grid_plot[:, 1],
    75,
    c="k",
    marker="o",
    label="Simulations",
    zorder=15,
)
axes[0].scatter(
    gw_class.grid_plot[:, 0],
    gw_class.grid_plot[:, 1],
    75,
    c="k",
    marker="o",
    label="Simulations",
    zorder=15,
)
iterations = np.array(gw_class.iter)
num_iter = len(np.unique(iterations))
grid_c = ["tab:gray", "tab:purple", "tab:olive", "tab:pink", "tab:brown"]
grid_i = 1
c = 1
for i in range(num_iter):
    if np.sum(iterations == i) == 6:
        temp_grid = gw_class.grid_plot[iterations == i, :2]
        start_point = np.array([np.min(temp_grid[:, 0]), np.min(temp_grid[:, 1])])
        end_point = np.array([np.max(temp_grid[:, 0]), np.max(temp_grid[:, 1])])
        wh = end_point - start_point
        axes[0].add_patch(
            Rectangle(
                start_point,
                wh[0],
                wh[1],
                alpha=1 / 3,
                facecolor=grid_c[grid_i - 1],
                edgecolor=None,
                label=f"Grid {grid_i:d}",
            )
        )
        grid_i += 1
    elif np.sum(iterations == i) == 1:
        axes[0].scatter(
            gw_class.grid_plot[iterations == i, 0],
            gw_class.grid_plot[iterations == i, 1],
            75,
            marker="o",
            label=f"Iteration {c:d}",
            zorder=20,
        )
        c += 1
axes[0].set_xlim([100, 900])
axes[0].set_xticks([200, 400, 600, 800])
axes[0].set_ylim([2, 21])
axes[0].set_yticks([4, 8, 12, 16, 20])
axes[0].scatter(
    gw_class.final_point[0],
    gw_class.final_point[1],
    25,
    c="r",
    marker="o",
    label=f"Converged = {np.round(gw_class.final_point[2],3):.3f} eV",
    zorder=20,
)

"""
LIF SECTION
"""
# index for LiF
idx_lif = idx_conv_plt[1]

# plot for the cs algorithm for LiF
gw_class = res_cs[idx_lif].gwc[-1]
axes[1].scatter(
    gw_class.grid[:, 0],
    gw_class.grid[:, 1],
    20,
    c="r",
    marker="x",
    zorder=1000,
)
axes[1].plot(gw_class.grid[:, 0], gw_class.grid[:, 1], "r-", linewidth=1, zorder=1000)
axes[1].scatter(
    gw_class.grid[-1, 0],
    gw_class.grid[-1, 1],
    75,
    c="r",
    marker="x",
    label=f"Converged = {np.round(gw_class.grid[-1,2],6):.3f} eV",
    zorder=20,
)
axes[1].set_xlabel(r"$N_\mathrm{b}$", fontsize=fs)
axes[1].xaxis.set_tick_params(labelsize=fs)
axes[1].yaxis.set_tick_params(labelsize=fs)
axes[1].tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=4,
    direction="in",
)
axes[1].tick_params(labelbottom=True, labeltop=False, labelleft=True, labelright=False)

# plot the npj algorithm for LiF
gw_class = res_npj[idx_lif].gwc[-1]
ref_gap = res_ref[idx_lif].gap[-1]
abs_error = 1000 * abs(gw_class.z_fit - ref_gap)
fit = axes[1].scatter(
    gw_class.x_fit,
    gw_class.y_fit,
    30,
    c=abs_error,
    marker="s",
    zorder=10,
    cmap="viridis_r",
)
cmap = plt.colorbar(fit, shrink=0.75, aspect=30, pad=0.05)
cmap.ax.tick_params(labelsize=8)
axes[1].scatter(
    gw_class.grid_plot[:, 0],
    gw_class.grid_plot[:, 1],
    75,
    c="k",
    marker="o",
    label="Simulations",
    zorder=15,
)
cmap.set_label(
    r"$\vert\Delta_\mathrm{R}F(N_\mathrm{b},\, G_\mathrm{cut})\vert$ (meV)",
    rotation=90,
    fontsize=fs,
)
iterations = np.array(gw_class.iter)
num_iter = len(np.unique(iterations))
grid_c = ["tab:gray", "tab:purple", "tab:brown"]
grid_i = 1
c = 1
for i in range(num_iter):
    if np.sum(iterations == i) == 6:
        temp_grid = gw_class.grid_plot[iterations == i, :2]
        start_point = np.array([np.min(temp_grid[:, 0]), np.min(temp_grid[:, 1])])
        end_point = np.array([np.max(temp_grid[:, 0]), np.max(temp_grid[:, 1])])
        wh = end_point - start_point
        axes[1].add_patch(
            Rectangle(
                start_point,
                wh[0],
                wh[1],
                alpha=0.5,
                facecolor=grid_c[grid_i - 1],
                edgecolor=None,
                label=f"Grid {grid_i:d}",
            )
        )
        grid_i += 1
    elif np.sum(iterations == i) == 1:
        axes[1].scatter(
            gw_class.grid_plot[iterations == i, 0],
            gw_class.grid_plot[iterations == i, 1],
            75,
            marker="o",
            label=f"Iteration {c:d}",
            zorder=20,
        )
        c += 1
axes[1].scatter(
    gw_class.final_point[0],
    gw_class.final_point[1],
    25,
    c="r",
    marker="o",
    label=f"Converged = {np.round(gw_class.final_point[2],3):.3f} eV",
    zorder=20,
)
axes[1].set_xlim([100, 1300])
axes[1].set_xticks([400, 800, 1200])
axes[1].set_ylim([2, 42])
axes[1].set_yticks([4, 12, 20, 28, 36])

# fix the layout
fig.tight_layout()
fig.subplots_adjust(wspace=0.25)

# save the figure
fig.savefig("plots/paper/Si_LiF.pdf")

Check how the W convergence depends on the k-point grid.

In [ ]:
# step size in our algorithm
delta_cs = [100, 4]
delta_npj = [200, 4]

# calculate the relative parameters change in W from the Gamma-only to final k-point grid
# normalized to the step size of the respective convergence algorithm
w_diff_cs = []
w_diff_npj = []
for r in res_cs:
    if not -1 in r.params[:, 0]:
        temp_w_diff_cs = (r.params[0, 1:] - r.params[-1, 1:]) / delta_cs
        w_diff_cs.append(temp_w_diff_cs)
    else:
        print(f"Skipping: {r.id} \t {r.name}")
    # for annotations
    if temp_w_diff_cs[0] < 0:
        print(f"CS Negative N_b:    {r.id:<10s} {r.name:<10s} {temp_w_diff_cs[0]}")
    if temp_w_diff_cs[1] < 0:
        print(f"CS Negative G_cut:  {r.id:<10s} {r.name:<10s} {temp_w_diff_cs[1]}")
for r in res_npj:
    if not -1 in r.params[:, 0]:
        temp_w_diff_npj = (r.params[0, 1:] - r.params[-1, 1:]) / delta_npj
        w_diff_npj.append(temp_w_diff_npj)
    else:
        print(f"Skipping: {r.id} \t {r.name}")
    # for annotations
    if temp_w_diff_npj[0] < 0:
        print(f"NPJ Negative N_b:   {r.id:<10s} {r.name:<10s} {temp_w_diff_npj[0]}")
    if temp_w_diff_npj[1] < 0:
        print(f"NPJ Negative G_cut: {r.id:<10s} {r.name:<10s} {temp_w_diff_npj[1]}")
w_diff_cs = np.array(w_diff_cs)
w_diff_npj = np.array(w_diff_npj)

In [ ]:
# figure size (tested)
width = 2 * 2.65
height = 4.5

# text size
fs = 10

# figure
fig, axs = plt.subplots(2, 2, figsize=(width, height), facecolor="w", dpi=600)

# add (a), (b), (c) and (d) to the subplots
axs[0, 0].text(-0.2, 1.15, r"\textbf{a}", transform=axs[0, 0].transAxes, size=11)
axs[0, 1].text(-0.18, 1.15, r"\textbf{b}", transform=axs[0, 1].transAxes, size=11)
axs[1, 0].text(-0.2, 1.15, r"\textbf{c}", transform=axs[1, 0].transAxes, size=11)
axs[1, 1].text(-0.18, 1.15, r"\textbf{d}", transform=axs[1, 1].transAxes, size=11)

# plot the change in the number of bands when comparing the gamma convergence to the last k-grid
axs[0, 0].hist(
    w_diff_cs[:, 0],
    bins=np.arange(np.min(w_diff_cs[:, 0]), np.max(w_diff_cs[:, 0]) + 5, 1) - 0.5,
    color="tab:blue",
)
axs[0, 0].axvline(x=np.mean(w_diff_cs[:, 0]), color="magenta", linestyle="--")
axs[0, 0].axvline(x=np.median(w_diff_cs[:, 0]), color="limegreen", linestyle="--")
axs[0, 1].hist(
    w_diff_npj[:, 0],
    bins=np.arange(np.min(w_diff_npj[:, 0]), np.max(w_diff_npj[:, 0]) + 5, 1) - 0.5,
    color="tab:blue",
)
axs[0, 1].axvline(x=np.mean(w_diff_npj[:, 0]), color="magenta", linestyle="--")
axs[0, 1].axvline(x=np.median(w_diff_npj[:, 0]), color="limegreen", linestyle="--")

# plot the change in the cutoff when comparing the gamma convergence to the last k-grid
axs[1, 0].hist(
    w_diff_cs[:, 1],
    bins=np.arange(np.min(w_diff_cs[:, 1]), np.max(w_diff_cs[:, 1]) + 5, 1) - 0.5,
    color="tab:blue",
)
axs[1, 0].axvline(x=np.mean(w_diff_cs[:, 1]), color="magenta", linestyle="--")
axs[1, 0].axvline(x=np.median(w_diff_cs[:, 1]), color="limegreen", linestyle="--")
axs[1, 1].hist(
    w_diff_npj[:, 1],
    bins=np.arange(np.min(w_diff_npj[:, 1]), np.max(w_diff_npj[:, 1]) + 5, 1) - 0.5,
    color="tab:blue",
)
axs[1, 1].axvline(x=np.mean(w_diff_npj[:, 1]), color="magenta", linestyle="--")
axs[1, 1].axvline(x=np.median(w_diff_npj[:, 1]), color="limegreen", linestyle="--")

# titles
axs[0, 0].set_title(r"CS", fontsize=fs + 2, pad=15)
axs[0, 1].set_title(r"SOTA", fontsize=fs + 2, pad=15)

# axis labels
axs[0, 0].set_xlabel(r"$C^\mathbf{\Gamma}_{N_\mathrm{b}}$", size=fs)
axs[0, 1].set_xlabel(r"$C^\mathbf{\Gamma}_{N_\mathrm{b}}$", size=fs)
axs[0, 0].set_ylabel(r"Occurrences", size=fs)
axs[1, 0].set_xlabel(r"$C^\mathbf{\Gamma}_{G_\mathrm{\scriptsize cut}}$", size=fs)
axs[1, 1].set_xlabel(r"$C^\mathbf{\Gamma}_{G_\mathrm{\scriptsize cut}}$", size=fs)
axs[1, 0].set_ylabel(r"Occurrences", size=fs)

# axis ticks
axs[0, 0].xaxis.set_tick_params(labelsize=fs)
axs[0, 0].yaxis.set_tick_params(labelsize=fs)
axs[0, 0].tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=4,
    direction="in",
)
axs[0, 0].tick_params(
    labelbottom=True, labeltop=False, labelleft=True, labelright=False
)
axs[0, 0].yaxis.set_ticks(axs[0, 0].get_yticks().astype(int))
axs[0, 0].xaxis.set_ticks([-3, -2, -1, 0, 1, 2, 3])
axs[0, 0].set_xlim([-4, 4])
axs[0, 1].xaxis.set_tick_params(labelsize=fs)
axs[0, 1].yaxis.set_tick_params(labelsize=fs)
axs[0, 1].tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=4,
    direction="in",
)
axs[0, 1].tick_params(
    labelbottom=True, labeltop=False, labelleft=True, labelright=False
)
axs[0, 1].yaxis.set_ticks(axs[0, 1].get_yticks().astype(int))
axs[0, 1].xaxis.set_ticks([-3, -2, -1, 0, 1, 2, 3])
axs[0, 1].set_xlim([-4, 4])
axs[1, 0].xaxis.set_tick_params(labelsize=fs)
axs[1, 0].yaxis.set_tick_params(labelsize=fs)
axs[1, 0].tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=4,
    direction="in",
)
axs[1, 0].tick_params(
    labelbottom=True, labeltop=False, labelleft=True, labelright=False
)
axs[1, 0].xaxis.set_ticks([-4, -2, 0, 2, 4, 6])
axs[1, 0].yaxis.set_ticks([0, 10, 20, 30])
axs[1, 0].set_xlim([-5, 7])
axs[1, 1].xaxis.set_tick_params(labelsize=fs)
axs[1, 1].yaxis.set_tick_params(labelsize=fs)
axs[1, 1].tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=4,
    direction="in",
)
axs[1, 1].tick_params(
    labelbottom=True, labeltop=False, labelleft=True, labelright=False
)
axs[1, 1].xaxis.set_ticks([-4, -2, 0, 2, 4, 6])
axs[1, 1].yaxis.set_ticks([0, 5, 10, 15, 20])
axs[1, 1].set_xlim([-5, 7])

# annotations
axs[0, 0].annotate(
    "NaF,\n" + r"BaF$_2$",
    xy=(-1, 1.5),
    xycoords="data",
    xytext=(-15, 25),
    textcoords="offset points",
    arrowprops=dict(arrowstyle="->"),
    fontsize=fs - 2,
)
axs[0, 0].annotate(
    "Xe",
    xy=(-2, 1.5),
    xycoords="data",
    xytext=(-5, 15),
    textcoords="offset points",
    arrowprops=dict(arrowstyle="->"),
    fontsize=fs - 2,
)
axs[0, 0].annotate(
    r"PbF$_2$",
    xy=(-3, 1.5),
    xycoords="data",
    xytext=(-10, 15),
    textcoords="offset points",
    arrowprops=dict(arrowstyle="->"),
    fontsize=fs - 2,
)
axs[0, 1].annotate(
    "Kr,\nKF,\nBeS,\nSrS,\nSrS,\nSrO,\nLiZnP",
    xy=(-1, 5),
    xycoords="data",
    xytext=(-15, 20),
    textcoords="offset points",
    arrowprops=dict(arrowstyle="->"),
    fontsize=fs - 2,
)
axs[1, 0].annotate(
    "Xe,\nAgCl",
    xy=(-1, 2),
    xycoords="data",
    xytext=(-15, 25),
    textcoords="offset points",
    arrowprops=dict(arrowstyle="->"),
    fontsize=fs - 2,
)
axs[1, 0].annotate(
    r"PbF$_2$",
    xy=(-2, 1),
    xycoords="data",
    xytext=(-15, 15),
    textcoords="offset points",
    arrowprops=dict(arrowstyle="->"),
    fontsize=fs - 2,
)
axs[1, 0].annotate(
    r"BaF$_2$",
    xy=(-4, 1),
    xycoords="data",
    xytext=(-9.5, 30),
    textcoords="offset points",
    arrowprops=dict(arrowstyle="->"),
    fontsize=fs - 2,
)
axs[1, 1].annotate(
    "ZnO,\nCuBr",
    xy=(-1, 2),
    xycoords="data",
    xytext=(-15, 30),
    textcoords="offset points",
    arrowprops=dict(arrowstyle="->"),
    fontsize=fs - 2,
)
axs[1, 1].annotate(
    "BeTe",
    xy=(-2, 1.5),
    xycoords="data",
    xytext=(-15, 20),
    textcoords="offset points",
    arrowprops=dict(arrowstyle="->"),
    fontsize=fs - 2,
)

# fix layout
fig.tight_layout()
fig.subplots_adjust(hspace=0.6, wspace=0.25)

# save the figure
fig.savefig("plots/paper/param_indep_w_k_1.pdf")

In [ ]:
# check the percentage of how many times the gamma-only grid is > 0,
# i.e. that the convergence surface is more flat on the final k-point grid compared to the gamma-only grid
print("CS: C_p >= 0")
print(
    f"{len(w_diff_cs[(w_diff_cs[:,0] >= 0) & (w_diff_cs[:,1] >= 0), 0]) / len(w_diff_cs[:,0]) * 100:.2f} %\n"
)
print("NPJ: C_p >= 0")
print(
    f"{len(w_diff_npj[(w_diff_npj[:,0] >= 0) & (w_diff_npj[:,1] >= 0), 0]) / len(w_diff_npj[:,0]) * 100:.2f} %\n"
)
# we need to check where C_P >= 0 holds for both parameters at the same time

# some statistics for the parameter stability of W with respect to the k-point grid density
print("CS: MEAN")
print(f"N_b:   {np.mean(w_diff_cs[:,0]):.1f}")
print(f"G_cut: {np.mean(w_diff_cs[:,1]):.1f}\n")

print("CS: MEDIAN")
print(f"N_b:   {np.median(w_diff_cs[:,0]):.1f}")
print(f"G_cut: {np.median(w_diff_cs[:,1]):.1f}\n")

print("CS: RANGE")
print(f"N_b:   {np.max(w_diff_cs[:,0]) - np.min(w_diff_cs[:,0]):.1f}")
print(f"G_cut: {np.max(w_diff_cs[:,1]) - np.min(w_diff_cs[:,1]):.1f}\n")

print("NPJ: MEAN")
print(f"N_b:   {np.mean(w_diff_npj[:,0]):.1f}")
print(f"G_cut: {np.mean(w_diff_npj[:,1]):.1f}\n")

print("NPJ: MEDIAN")
print(f"N_b:   {np.median(w_diff_npj[:,0]):.1f}")
print(f"G_cut: {np.median(w_diff_npj[:,1]):.1f}\n")

print("NPJ: RANGE")
print(f"N_b:   {np.max(w_diff_npj[:,0]) - np.min(w_diff_npj[:,0]):.1f}")
print(f"G_cut: {np.max(w_diff_npj[:,1]) - np.min(w_diff_npj[:,1]):.1f}")

Check how the k-point grid convergence depends on the W parameters.

In [ ]:
# for the k-point convergence we analyze the difference in the gaps
# on different k-point meshes calculated with starting and reference parameters in W
# we check at which k-point grid the gap is converged (difference < 25 meV)
conv_thr = 25  # meV (we define this again to be save)
mat_k_conv = []
counter = -1
for i in range(len(res_cs)):
    min_gaps = np.zeros(5)
    for j in range(5):
        min_gaps[j] = res_cs[i].gwc[j].grid[0, 2]
    ref_gaps = res_ref[i].gap
    gap_diff_min = np.abs(np.diff(min_gaps)) * 1000  # meV
    gap_diff_max = np.abs(np.diff(ref_gaps)) * 1000  # meV
    bool_min = gap_diff_min <= conv_thr
    bool_max = gap_diff_max <= conv_thr
    if any([bool_min[-1] and bool_max[-1]]):
        mat_k_conv.append(np.argmax(bool_min) - np.argmax(bool_max))
        counter += 1
    # in this case we assume that the convergence happens on the next grid
    # this is likely for GaAs (we check the difference in the gaps)
    elif any([bool_min[-1] or bool_max[-1]]):
        print("\nOne did not converge!: ", bool_min[-1], bool_max[-1], res_cs[i].name)
        if not bool_min[-1]:
            temp_min = 3
            temp_max = np.argmax(bool_max)
        if not bool_max[-1]:
            temp_min = np.argmax(bool_min)
            temp_max = 4
        mat_k_conv.append(temp_min - temp_max)
        counter += 1
    else:
        print("\nBoth did not converge: ", bool_min[-1], bool_max[-1], res_cs[i].name)
    if mat_k_conv:
        if mat_k_conv[counter] != 0:
            print("\nDifferent convergence points:", res_cs[i].name)
            print(gap_diff_min)
            print(gap_diff_max)
mat_k_conv = np.array(mat_k_conv)

In [ ]:
# check if the k-point grid convergence is the same for minimal and maximal parameters in W
print(
    f"Number of times where it did not converge at the same point: {len(mat_k_conv[mat_k_conv > 0]) + len(mat_k_conv[mat_k_conv < 0]):d}"
)
print(
    f"Number of times where the convergence happend to late with minimal W parameters: {len(mat_k_conv[mat_k_conv > 0]):d}"
)
print(
    f"Number of times where the convergence happend to early with minimal W parameters: {len(mat_k_conv[mat_k_conv < 0]):d}\n"
)

# same number as percentages
print(
    f"Number of times where it did not converge at the same point: {(len(mat_k_conv[mat_k_conv > 0]) + len(mat_k_conv[mat_k_conv < 0])) / len(mat_k_conv) * 100:.2f} %"
)
print(
    f"Number of times where the convergence happend to late with minimal W parameters: {len(mat_k_conv[mat_k_conv > 0]) / len(mat_k_conv) * 100:.2f} %"
)
print(
    f"Number of times where the convergence happend to early with minimal W parameters: {len(mat_k_conv[mat_k_conv < 0])/ len(mat_k_conv) * 100:.2f} %"
)

In [ ]:
# we investigate the average speedup by converging the
# W parameters on a gamma-only calculation compared to the 2x2x2 k-point grid
# proposed by van Setten et al.
w_speed_up = []
for r in res_cs:
    w_speed_up.append(np.sum(r.gwc[1].grid_time) / np.sum(r.gwc[0].grid_time))
print(
    f"Average speed up for the W parameter convergence on a gamma-only grid: {np.mean(w_speed_up):.1f}"
)
print(
    f"Median speed up for the W parameter convergence on a gamma-only grid: {np.median(w_speed_up):.1f}"
)

In [ ]:
# we investigate the average speedup by converging the
# k-grid with low W parameters compared with the W parameters from the CS convergence
# we note that this is just a approximation as the W parameters at the last point of
# the CS convergence algorithm may change with the k-point grid, however by looking
# at C_p we know that this happens in about 10 % of the cases
# however, by looking at C_P for different k-point grids this effect should average out,
# since the distribution of C_P is roughly symmetric around zero.
k_speed_up = []
for r in res_cs:
    temp_t_min = 0
    temp_t_cs = 0
    for g in r.gwc:
        temp_t_min += g.grid_time[0]
        temp_t_cs += g.grid_time[-1]
    k_speed_up.append(temp_t_cs / temp_t_min)
print(
    f"Average speed up for k-grid convergence with low W parameters: {np.mean(k_speed_up):.1f}"
)
print(
    f"Median speed up for k-grid convergence with low W parameters: {np.median(k_speed_up):.1f}"
)